## Demo of the Adaptive Ground Point Filtering Library

This notebook gives a tour of the capabilities of `filteradapt`. It is continuously updated while we develop the library. You can execute the code in each cell by pressing `Ctrl+Enter`. To start your own project based on `filteradapt`, you can either edit this notebook or start a new one.

The first thing to do in a Jupyter notebook that uses `filteradapt` is to import the library:

In [ ]:
import adaptivefiltering

The next step you will typically do is to construct a dataset object by reading a LAS/LAZ file:

In [ ]:
ds = adaptivefiltering.DataSet(filename="data/500k_NZ20_Westport.laz")

The data set can be visualized in the notebook through its `show()` method:

In [ ]:
ds.show()

In [ ]:
ds.save_mesh("test", resolution=5)


In [ ]:
ds.show_mesh(resolution=3)